## Deep Learning Approaches for RF-based detection & classification

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

# import the torch packages
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

import torchvision.models as models

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# import custom functions
from helper_functions import *
from latency_helpers import *
from loading_functions import *

from sklearn import preprocessing

### Load Features

In [2]:
feat_folder = '../Features/'
feat_name = 'SPEC'
seg_len = 20
# datestr = '2022-07-05'
n_per_seg = 256
interferences = ['CLEAN']
dataset = load_dronedetect_data(feat_folder, feat_name, seg_len, n_per_seg, interferences)

100%|███████████████████████████████████████████| 80/80 [01:36<00:00,  1.21s/it]


In [3]:
print('dataset size', len(dataset))
print('shape of each item', dataset.__getitem__(10)[0].shape)

dataset size 9487
shape of each item torch.Size([1, 129, 4687])


## Transfer learning from Resnet50 & Apply Logistic Regression (Swinney paper)

In [19]:
# use pretrained resnet feature and just keep up to the last layer
resnet50 = models.resnet50(pretrained=True)
modules=list(resnet50.children())[:-1]
# resnet50=nn.Sequential(*modules)
for p in resnet50.parameters():
    p.requires_grad = False

In [20]:
# test resnet
input = torch.randn(1,1,30,300)
inputr = input.repeat(1,3,1,1)
resnet50(inputr).shape

torch.Size([1, 1000])

In [21]:
resnet_feats = []
resnet_y = []
for n in range(len(dataset)):
    d = dataset.__getitem__(n)
    inarr = d[0]
    inputr = inarr.repeat(1,3,1,1)  # repeat to have 3 channels of the same info
    out = resnet50(inputr)
    resnet_feats.append(np.array(out))
    resnet_y.append(np.array(d[1]))

resnet_feats = np.array(resnet_feats)
resnet_y = np.array(resnet_y)

# flatten the middle dimension
resnet_feats = resnet_feats.reshape(resnet_feats.shape[0], resnet_feats.shape[-1])
# invert labels back to categorical
resnet_y_cat = dataset.le.inverse_transform(resnet_y.astype(np.int64))

## Transfer learning from VGG 19

In [5]:
# import torch
vgg19 = torch.hub.load('pytorch/vision:v0.10.0', 'vgg19', pretrained=True)
modules=list(vgg19.children())[:-1]
# resnet50=nn.Sequential(*modules)
for p in vgg19.parameters():
    p.requires_grad = False

Using cache found in /home/kzhou/.cache/torch/hub/pytorch_vision_v0.10.0


In [7]:
vgg_feats = []
vgg_y = []
for n in tqdm(range(len(dataset))):
    d = dataset.__getitem__(n)
    inarr = d[0]
    inputr = inarr.repeat(1,3,1,1)  # repeat to have 3 channels of the same info
    out = vgg19(inputr)
    vgg_feats.append(np.array(out))
    vgg_y.append(np.array(d[1]))

vgg_feats = np.array(vgg_feats)
vgg_y = np.array(vgg_y)

# flatten the middle dimension
vgg_feats = vgg_feats.reshape(vgg_feats.shape[0], vgg_feats.shape[-1])
# invert labels back to categorical
vgg_y_cat = dataset.le.inverse_transform(vgg_y.astype(np.int64))

### TO DO: Save these features to be easily loaded [ this took almost 2 hours]

100%|█████████████████████████████████████| 9487/9487 [1:54:38<00:00,  1.38it/s]


In [49]:
feats_lr = vgg_feats # which features to use for logit reg
y_cat = vgg_y_cat

In [ ]:
# split data into K-fold
k_fold = 5
cv = KFold(n_splits=k_fold, random_state=1, shuffle=True)

# model parameters
Cs=list(map(lambda x:pow(2,x),range(-2,10,2)))

best_params_ls = []
acc_ls = []
f1_ls = []
runt_ls = []

parameters = {'C':Cs}

for train_ix, test_ix in tqdm(cv.split(feats_lr)):
    
    # find the optimal hypber parameters
    lr = LogisticRegression(max_iter=10000)
    clf = GridSearchCV(lr, parameters, n_jobs=1)
    clf.fit(feats_lr[train_ix], y_cat[train_ix])
    
    print(clf.best_params_)
    best_params_ls.append(clf.best_params_)
    
    # predict on the test data
    y_pred, runtimes = atomic_benchmark_estimator(clf, feats_lr[test_ix], verbose=False)
    runt_ls.append(np.mean(runtimes))
    
    acc = accuracy_score(y_arr[test_ix], y_pred)
    f1 = f1_score(y_arr[test_ix], y_pred, average='weighted')
    print('Accuracy: {:.3},\t F1: {:.3}'.format(acc,f1))
    acc_ls.append(acc)
    f1_ls.append(f1)
    
out_msg = feat_name+': ResNet+LR average test acc: {:.2}, F1: {:.2}, Run-time: {:.2}ms'.format(np.mean(acc_ls), np.mean(f1_ls), np.mean(runt_ls)*1e3)
print(out_msg)

0it [00:00, ?it/s]

## 3. Apply resnet & a fully connected layer

In [ ]:
# # Define network
# model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg19', pretrained=True)
# model.classifier = nn.Linear(model.classifier[0].in_features, num_classes)
# print(model)

In [33]:
class ResnetFC(nn.Module):
    def __init__(self, num_classes):
        super(ResnetFC,self).__init__()
        self.num_classes = num_classes
        self.resnet = models.resnet50(pretrained=True)
        
        for param in self.resnet.parameters():
            self.resnet.requires_grad_(False)
        
        self._fc = nn.Linear(1000, num_classes)
    def forward(self, x):
#         batch_size ,_,_ =x.shape
        
        # replicate the image to have 3 channels
        x = x.repeat(1,3,1,1)
        x = self.resnet(x)
        x = self._fc(x)
        
        return x

In [34]:
fctest = ResnetFC(7)

In [35]:
fctest.forward(dataset.__getitem__(10)[0])

tensor([[-0.0363, -1.0948,  0.0186,  0.1136, -0.1828,  0.1280, -0.2705]],
       grad_fn=<AddmmBackward0>)

In [96]:
# # make a dataset out of resnet features
# class ResNetFeatData(Dataset): ## NUMBERICAL DATA
#     def __init__(self, Xarr, yarr):
#         self.Xarr = Xarr
#         test_list=[]
#         self.le = preprocessing.LabelEncoder()
#         self.le.fit(yarr.flatten())
#         self.yarr = le.transform(yarr.flatten())
        
#     def __len__(self):
#         return len(self.yarr)
    
#     def __getitem__(self, index):
#         # all data must be in float and tensor format
#         X = torch.tensor((self.Xarr[index]))
#         X = X.unsqueeze(0)
#         y = torch.tensor(float(self.yarr[index]))
#         return (X, y)

In [97]:
# resdataset = ResNetFeatData(resnet_feats, y_arr)

In [5]:
# ## Network for fully connected layer
# class FCNet(nn.Module):
#     def __init__(self, num_classes):
#         super(FCNet, self).__init__()
#         self.fc_layer = nn.Linear(1000, num_classes)
    
#     # Progresses data across layers    
#     def forward(self, x):
#         x = self.fc_layer(x)
#         return x


### development code - function for cross validation for any model

In [44]:
def runkfoldcv(model, dataset, device, k_folds, batch_size, learning_rate, num_epochs, momentum, l2reg):
    num_classes = model.num_classes
    
    # For fold results
    results = {}
    runtimes = np.zeros(k_folds)
    f1s = np.zeros(k_folds)

    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=k_folds, shuffle=True)

    # Start print
    print('--------------------------------')

    # K-fold Cross Validation model evaluation
    for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
        # Print
        print(f'FOLD {fold}')
        print('--------------------------------')

        # Sample elements randomly from a given list of ids, no replacement.
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

        # Define data loaders for training and testing data in this fold
        trainloader = torch.utils.data.DataLoader(
                          dataset, 
                          batch_size=batch_size, sampler=train_subsampler)
        testloader = torch.utils.data.DataLoader(
                          dataset,
                          batch_size=batch_size, sampler=test_subsampler)

        # Init the neural network
        model = model.to(device)
    #     network.apply(reset_weights)

        criterion = nn.CrossEntropyLoss()

        # Initialize optimizer
    #     optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=l2reg, momentum = momentum)  

        # Run the training loop for defined number of epochs
        for epoch in range(0, num_epochs):
            # Print epoch
            print(f'Starting epoch {epoch+1}')

            # Set current loss value
            current_loss = 0.0

            # Iterate over the DataLoader for training data
            for i, data in enumerate(trainloader):
                # Get inputs
                inputs, targets = data
                inputs = inputs.float()
#                 inputs = torch.squeeze(inputs, 1)
                targets= targets.type(torch.long)

                # Move tensors to configured device
                inputs = inputs.to(device)
                targets = targets.to(device)
                
                

                # Perform forward pass
                outputs = model(inputs)

                # Compute loss            
                loss = criterion(outputs, targets)

                # Zero the gradients
                optimizer.zero_grad()

                # Perform backward pass
                loss.backward()

                # Perform optimization
                optimizer.step()

                # Print statistics
                current_loss += loss.item()
                if i % 50 == 49:
                    print('    Loss after mini-batch %5d: %.5f' %
                          (i + 1, current_loss / 50))
                    current_loss = 0.0
    #         print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

        # Process is complete.
    #     print('Training process has finished. Saving trained model.')

        # Print about testing
        print('Starting testing')
        print('----------------')

        # Saving the model
    #     save_path = f'./model-fold-{fold}.pth'
    #     torch.save(network.state_dict(), save_path)

        # Evaluation for this fold
        correct, total = 0, 0
        network.eval()
        with torch.no_grad():
            starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
            runtimes_thisfold = []
            f1s_thisfold = []
            # Iterate over the test data and generate predictions
            for i, data in enumerate(testloader, 0):
                # Get inputs
                inputs, targets = data
                inputs = inputs.to(device)
                targets = targets.to(device)

                # Generate outputs
                n_instances = len(inputs)
                ys = torch.empty(n_instances)
                ys = ys.to(device)

                for i in range(n_instances):
                    instance = inputs[i]
                    instance = instance.float()
                    start = time.time()
                    starter.record()
                    yi = model(instance)
                    _,pred = torch.max(yi,1)
                    ender.record()

                    torch.cuda.synchronize()
                    curr_time = starter.elapsed_time(ender) #miliseconds

                    runtimes_thisfold.append(curr_time*1e-3)
                    ys[i] = pred


                # Set total and correct
                total += targets.size(0)
                correct += (ys == targets).sum().item()
                f1i = f1_score(ys.cpu().numpy(), targets.cpu().numpy())
                f1s_thisfold.append(f1i)

            mean_runtime = np.mean(np.array(runtimes_thisfold))
            mean_f1 = np.mean(np.array(f1s_thisfold))

        # Summarize and print results
        results[fold] = 100.0 * (correct / total)
        runtimes[fold] = mean_runtime
        f1s[fold] = mean_f1
        print('Accuracy for fold %d: %.2f %%' % (fold, 100.0 * correct / total))
        print('F1 for fold %d: %.2f ' % (fold, mean_f1))
        print('Runtime for fold %d: %.4f s' % (fold, mean_runtime))
        print('--------------------------------')

    # Print fold results
    print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
    print('--------------------------------')
    sum = 0.0
    for key, value in results.items():
        print(f'Fold {key}: {value} %')
        sum += value
    mean_acc = sum/len(results.items())
    mean_f1s = np.mean(f1s)
    mean_runtime = np.mean(runtimes)
    print(f'Average Accuracy: {mean_acc} %')
    print(f'Average F1: {mean_f1s}')
    print(f'Average Runtime: {mean_runtime} s')
    
    return avg_acc, mean_f1s, mean_runtime

In [ ]:
# Configuration options
k_folds = 2

batch_size = 8 # 128
num_classes = 7
learning_rate = 0.01
num_epochs = 1 # 0
momentum = 0.95
l2reg = 1e-4

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [46]:
runkfoldcv(fctest, dataset, device, k_folds, batch_size, learning_rate, num_epochs, momentum, l2reg)

--------------------------------
FOLD 0
--------------------------------
Starting epoch 1


RuntimeError: CUDA out of memory. Tried to allocate 4.65 GiB (GPU 0; 23.65 GiB total capacity; 11.67 GiB already allocated; 2.77 GiB free; 11.69 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF